In [119]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd
import cv2
import argparse
import mediapipe as mdp
import copy
import os
import csv
from model import KeyPointClassifier
import itertools

ImportError: cannot import name 'KeyPointClassifier' from 'model' (unknown location)

In [ ]:
#GETS THE REQUIRED ARGUMENTS IN CLI 

def get_args():
    parser=argparse.ArgumentParser()

    parser.add_argument("--device", type= int,default=0)
    parser.add_argument("--staticimage", action="store_true")
    parser.add_argument("--width",help="Maximun width", type= int,default=960)
    parser.add_argument("--height",help="Maximun height", type= int,default=540)

    parser.add_argument("--minDetection" ,help="Minimum confidence for detection",type=float,default=0.6)
    parser.add_argument("--minTracking" ,help="Minimum confidenxce for tracking",type=float,default=0.8)

    args = parser.parse_args()

    return args


In [ ]:
def load_models(use_static_image_mode,min_detection,min_tracking):

#LOAD MODEL TO DETECT LANDMARKS IN HANDS
    mdp_hands=mdp.solutions.hands
    hands=mdp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_num_hands=2,
        min_detection_confidence=min_detection,
        min_tracking_confidence=min_tracking
        )
    return hands

In [ ]:
def camera_input(cam_device,cam_width,cam_height):

#POPERLY INITIALIZE CAMERA FOR WORKING
    cam=cv2.VideoCapture(cam_device)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH,cam_width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT,cam_height)
    return cam



In [ ]:
# NOW TO FIND THE EXACT COORDINATES IN THE SCREEN AS THE IMAGES ARE NORMALIZED RIGHT NPW

def screen_coordinates(normalized_coodr,image):
    img_height,img_width,_=image.shape

    screen_coordinate=[]

    for _,landmark in enumerate(normalized_coodr.landmark):
        screen_x=int(landmark.x*img_width)
        screen_y=int(landmark.y*img_height)
        #NO NEED FOR LANDMARK Z AND ALSO IMAGE.SHAPE[2] PROVIDES COLOR INDEXES LIKE RGB SO 3
        screen_coordinate.append([screen_x,screen_y])
        cv2.circle(image, (screen_x, screen_y), 5, (0, 255, 0), -1)

    return screen_coordinate

In [ ]:
#NORMALIZATION SO THAT WE CAN DO SOMETHING EVEN WHEN THE HAND IS MOVED AND IT STILL WORKS ON THE HAND GEUSTURE
#THE NORMALIZED POINTS MAKES IT SO THA WE NEED LESSER DATA AND SIMPLER MODEL TO TRAIN AND DETECT

def normalized_values(screen_coordinate):

    temp_landmark=copy.deepcopy(screen_coordinate)
    base_x,base_y=0,0

    print(type(temp_landmark))

    for i, pixel_val in enumerate(temp_landmark):

        if i==0:
            base_x,base_y=screen_coordinate[0][0],screen_coordinate[0][1]
        
        temp_landmark[i][0]=pixel_val[0]-base_x
        temp_landmark[i][1]=pixel_val[1]-base_y

    # temp_landmark=np.array(temp_landmark).flatten()
    temp_landmark = list(itertools.chain.from_iterable(temp_landmark))
    max_val=max(list(map(abs,temp_landmark)))

    def normalize_(val):
        return val/max_val
    
    temp_landmark=list(map(normalize_,temp_landmark))

    return temp_landmark
    

        

In [ ]:
# NOW TO STORE THE DATAS IN A CSV FILE 
def store_data(number,normalized_one):
    csv_path=r"datas/pre_processed.csv"
    
    with open(csv_path,'a',newline="") as send:
        store=csv.writer(send)
        store.writerow([number,*normalized_one])
    
    return


In [ ]:
def image_path(path=''):
    paths=[]
    path_list=[]
    pathx=[]
    if not path:
        path=r"datas\photos_hands"
    all_files=list(map(int,os.listdir(path)))

    for i in range(len(all_files)):
        paths.append(os.path.join(path,str(all_files[i])))
    for path in paths:
        path_list=[]
        for i in os.listdir(path):
            if i.endswith('.png'):
                path_list.append(os.path.join(path,i))
        pathx.append(list(path_list))
        
    return pathx,all_files

image_path()


([['datas\\photos_hands\\1\\image copy.png',
   'datas\\photos_hands\\1\\image.png'],
  ['datas\\photos_hands\\2\\image.png']],
 [1, 2])

In [ ]:
image_path()


([['datas\\photos_hands\\1\\image copy.png',
   'datas\\photos_hands\\1\\image.png'],
  ['datas\\photos_hands\\2\\image.png']],
 [1, 2])

In [ ]:
def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

In [ ]:
def calc_boundrect(image,coodr):
    x_val=[(i[0]) for i in coodr]
    y_val=[(i[1]) for i in coodr]
    x=min[x_val]
    y=min[y_val]
    w=max[x_val]-x
    h=max[y_val]-y
    return [x,y,x+w,y+h]

In [ ]:
def draw_landmarks(image,results,hands,screen_coordinate):
    mdp_hands = mdp.solutions.hands
# Check if any hands are detected.
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw the landmarks on the image.
            mdp.solutions.drawing_utils.draw_landmarks(
                image, hand_landmarks, mdp_hands.HAND_CONNECTIONS,
                mdp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                mdp.solutions.drawing_styles.get_default_hand_connections_style()
            )

    # Display the image with landmarks.
    cv2.imshow('Hand Landmarks', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
def main():

# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    use_static_image_mode=True
    min_detection=0.6
    min_tracking=0.8


    hands=load_models(use_static_image_mode,min_detection,min_tracking)
    paths,num=image_path()
    idx=0
    print(paths)
    for path in paths:
        for i in path:
            image=cv2.imread(i)
            image = cv2.flip(image, 1)
            # image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            final_img=copy.deepcopy(image)
            
            image.flags.writeable = False
            results = hands.process(image)
            image.flags.writeable = True
    
            screen_coordinate=screen_coordinates(results.multi_hand_landmarks[0],final_img)
           
            normalized_coodr=normalized_values(screen_coordinate)

            final_img=draw_landmarks(final_img,results,hands,screen_coordinate)
            store_data(num[idx],normalized_coodr)
          
          
        idx=+1
        cv2.destroyAllWindows()
main()

[['datas\\photos_hands\\1\\image copy.png', 'datas\\photos_hands\\1\\image.png'], ['datas\\photos_hands\\2\\image.png']]
<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:
def draw_bounding_rect(image,rect):
    cv2.rectangle(image, (rect[0], rect[1]), (rect[2], rect[3]),(0, 0, 0), 1)
    return image

In [ ]:
simpleclassifier=KeyPointClassifier()

NameError: name 'KeyPointClassifier' is not defined

In [ ]:
def main():
    # GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    cam_device=0
    cam_width=960
    cam_height=540
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    use_static_image_mode=True
    min_detection=0.6
    min_tracking=0.8

    cam=camera_input(cam_device,cam_width,cam_height)
    hands=load_models(use_static_image_mode,min_detection,min_tracking)
    mdp_hands = mdp.solutions.hands
  

    simpleclassifier=SimpleClassifier()
    
    while True:
        # Process Key (ESC: end) #################################################
        key = cv2.waitKey(10)
        if key == 27:  # ESC
            break
        

        # Camera capture #####################################################
        ret, image = cam.read()
        if not ret:
            break
        image = cv2.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)

        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True

        if results.multi_hand_landmarks: 
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,results.multi_handedness):
                # print(results.multi_hand_landmarks[0])
                screen_coordinate=screen_coordinates(results.multi_hand_landmarks[0],image)
                normalized_coodr=normalized_values(screen_coordinate)
                classification_id = simpleclassifier(normalized_coodr)
                print(classification_id)
                bound_rect=calc_bounding_rect(debug_image,results.multi_hand_landmarks[0])
                
                for hand_landmarks in results.multi_hand_landmarks:
                    # Draw the landmarks on the image.
                    mdp.solutions.drawing_utils.draw_landmarks(
                    debug_image, hand_landmarks, mdp_hands.HAND_CONNECTIONS,
                    mdp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                    mdp.solutions.drawing_styles.get_default_hand_connections_style()
                )
                    
                debug_image = draw_bounding_rect(debug_image, bound_rect)

        cv2.imshow('Hand Gesturet', debug_image)

    cam.release()
    cv2.destroyAllWindows()
main()



TypeError: 'module' object is not callable

In [ ]:
video=cv2.VideoCapture('')

In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Load the image.
image = cv2.imread(r'C:\Users\sinju\Documents\HandsDetectSign\Hand_gesture_detection\datas\photos_hands\1\image copy.png')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width, _ = image.shape

# Process the image to find hands.
results = hands.process(image_rgb)

# Check if any hands are detected.
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        # Convert normalized coordinates to pixel coordinates.
        for landmark in hand_landmarks.landmark:
            x_px = int(landmark.x * width)
            y_px = int(landmark.y * height)
            # Draw a circle at each landmark.
            cv2.circle(image, (x_px, y_px), 5, (0, 255, 0), -1)

# Display the image with landmarks.
cv2.imshow('Hand Landmarks', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
cam_device=0
cam_width=960
cam_height=540



In [ ]:
def camerastuff(cam_device,cam_width,cam_height):
    cam=cv2.VideoCapture(cam_device)
    while True:
        cam.set(cv2.CAP_PROP_FRAME_WIDTH,cam_width)
        cam.set(cv2.CAP_PROP_FRAME_HEIGHT,cam_height)
        # Process Key (ESC: end) #################################################
        key = cv2.waitKey(10)
        if key == 27:  # ESC
            break
        

        # Camera capture #####################################################
        ret, image = cam.read()
        if not ret:
            break
        image = cv2.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)

        cv2.imshow('Hand Gesture Recognition', image)

    cam.release()
    cv2.destroyAllWindows()



In [ ]:
csv_path="../datas/pre_processed.csv"
folder_path="../datas/photos_hands"
all_num=list(map(int,(os.listdir(folder_path))))

all_num

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../datas/photos_hands'

In [ ]:
x=results.multi_hand_landmarks
for i,val in enumerate(x[0].landmark):
    print(val.z)

In [ ]:
# Example function to process hand landmarks
def process_hand_landmarks(hand_landmarks: landmark_pb2.NormalizedLandmarkList):
    for idx, landmark in enumerate(hand_landmarks.landmark):
        print(f'Landmark {idx}: x={landmark.x}, y={landmark.y}, z={landmark.z}')

# Example usage with dummy data
example_landmarks = landmark_pb2.NormalizedLandmarkList()
example_landmarks.landmark.add(x=0.5, y=0.5, z=0.5)
example_landmarks.landmark.add(x=0.6, y=0.6, z=0.6)

process_hand_landmarks(example_landmarks)

In [ ]:
results=main()
output_dict = results.__dict__

# Print the entire dictionary
for key, value in output_dict.items():
    print(f"{key}: {value}")